In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import tensorflow as tf
path = r'E://'
id_dict = {}
word_dict = {}

emb_name = "NonDynCleanDeleteBaseline_150w"

In [2]:
#AutoDetect dims
#test with base2021_300
with open(path + emb_name, 'r' , encoding="utf-8")  as file:
    line0 = file.readline()
    dimensions = len(line0.split())-1
print("dimensions "+str(dimensions))

dimensions 500


In [3]:
matrix = []
with open(path + emb_name, 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        entry = line.split()
        word = entry[0].strip()
        values = entry[1:]
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values, "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

260377


In [4]:
def find_candidate(sim,exclusion_list_ids,isMult):
    for exclude_id in exclusion_list_ids:
        if isMult:
             sim[exclude_id] = 0#lowest possible value in 3CosMult
        else:
            sim[exclude_id] = -1#lowest possible value in 3CosAdd
    return np.argmax(sim)

In [5]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)# only use normalised version !!!
matrix = []

In [ ]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list

def unzip(some_list):
    return [ i for i, j in some_list ]
    
# take nearest 6 and remove the searched word itself
print(unzip(find_nearest_k('bank' ,7)[1:]))#the food or the place
print(unzip(find_nearest_k('apple',7)[1:]))#location or to to speak to
print(unzip(find_nearest_k('so'   ,7)[1:]))#noun (animal) or verb 
print(unzip(find_nearest_k('desert',   6)[1:]))#animal or verb
print(unzip(find_nearest_k('left',   6)[1:]))#adverb (direction) and verb(the plane left)
print(unzip(find_nearest_k('duck',   6)[1:]))# The president of the bank walked along the river bank.
print("--------")

# 3cosAdd normalised befor arithmetic

In [6]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim.numpy(),[id_a,id_a_star,id_b],False)
    return word_dict[index]

## 3cos Mult

In [7]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim.numpy(),[id_a,id_a_star,id_b],True)
    return word_dict[index]

# Test all

In [8]:
def format_results(add,mult):
    add_string = str(add)
    add_string = add_string.lstrip('0')
    add_string = format(add_string, ".4")
    mult_string = str(mult)
    mult_string = mult_string.lstrip('0')
    mult_string = format(mult_string, ".4")
    return add_string +'/'+mult_string


In [9]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0]
count_syn_sucess      = [0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    if file_name.endswith('.classes'):
        continue
    
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:    
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
         
        for line_id,line in enumerate(lines):
            if (line_id % 10 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_sem_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[0] += 1
                    count_syn_sucess[0] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
        print(format_results(local_sucesses[0]/float(local_tasks_count),local_sucesses[1]/float(local_tasks_count)))
        print('-------------------')

print('semantical')
print(format_results(count_sem_sucess[0]/float(count_sem_questions),count_sem_sucess[1]/float(count_sem_questions)))

print('syntactical')
print(format_results(count_syn_sucess[0]/float(count_sem_questions),count_syn_sucess[1]/float(count_sem_questions)))

print('overall')
print(format_results((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions), \
                     (count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions)))

capital-common-countries.txt
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
.934/.934
-------------------
capital-world.txt
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1

170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
.592/.587
-------------------
gram9-plural-verbs.txt
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
.262/.257
-------------------
ner.classes
treebank_pos.classes
universal_pos.classes
semantical
.775/.774
sy

In [10]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [11]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            print(id1)
            print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
58274
34268
tf.Tensor(0.9948665165286512, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
58274
58274
tf.Tensor(1.0, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
202953
35611
tf.Tensor(0.9945349118206713, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
156986
35611
tf.Tensor(0.996950317225267, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
139248
176699
tf.Tensor(0.9975300707292811, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
118685
176699
tf.Tensor(0.9972227315311613, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
219262
74084
tf.Tensor(0.9919743087415465, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
64788
185448
tf.Tensor(0.9737629400256691, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
234074
107922
tf.Tensor(0.9953666977395734, shape=(), dtype=float64) 0.7
lineprofessor	doctor	6.

tf.Tensor(0.9901948890446421, shape=(), dtype=float64) 0.7809999999999999
linelobster	wine	5.70

lobster wine
42032
5668
tf.Tensor(0.9888609749434425, shape=(), dtype=float64) 0.5700000000000001
linechampionship	tournament	8.36

championship tournament
97374
161120
tf.Tensor(0.9969243464765138, shape=(), dtype=float64) 0.836
lineman	woman	8.30

man woman
108231
82521
tf.Tensor(0.9985360444484157, shape=(), dtype=float64) 0.8300000000000001
lineman	governor	5.25

man governor
108231
34291
tf.Tensor(0.9950961102202455, shape=(), dtype=float64) 0.525
linemurder	manslaughter	8.53

murder manslaughter
41798
16982
tf.Tensor(0.9828662558136521, shape=(), dtype=float64) 0.853
lineopera	performance	6.88

opera performance
190922
126124
tf.Tensor(0.9956332016512035, shape=(), dtype=float64) 0.688
lineMexico	Brazil	7.44

Mexico Brazil
0 0.744
lineglass	metal	5.56

glass metal
217762
38702
tf.Tensor(0.9960296331239658, shape=(), dtype=float64) 0.5559999999999999
linealuminum	metal	7.83

aluminum m

tf.Tensor(0.9940840903724233, shape=(), dtype=float64) 0.3
lineprejudice	recognition	3.00

prejudice recognition
230329
192098
tf.Tensor(0.9916047857454425, shape=(), dtype=float64) 0.3
lineviewer	serial	2.97

viewer serial
244326
180445
tf.Tensor(0.9883580681688489, shape=(), dtype=float64) 0.29700000000000004
linepeace	insurance	2.94

peace insurance
223787
52509
tf.Tensor(0.99350487727538, shape=(), dtype=float64) 0.294
lineMars	water	2.94

Mars water
0 0.294
linemedia	gain	2.88

media gain
118685
179236
tf.Tensor(0.9967117219660806, shape=(), dtype=float64) 0.288
lineprecedent	cognition	2.81

precedent cognition
164348
75022
tf.Tensor(0.9797843158485243, shape=(), dtype=float64) 0.281
lineannouncement	effort	2.75

announcement effort
155882
74221
tf.Tensor(0.9965777544810096, shape=(), dtype=float64) 0.275
lineline	insurance	2.69

line insurance
199887
52509
tf.Tensor(0.9941771550492342, shape=(), dtype=float64) 0.269
linecrane	implement	2.69

crane implement
88074
121933
tf.Tensor

In [12]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.15898419513558842, pvalue=0.02347480205593653)